# Week 4 - Practice Exercises

This notebook contains a range of practice exercises that demonstrate how data transformation and geoprocessing operations can be sequenced to complete a task. Here, we have a vector dataset of polygon geometries which correspond to areas in Fiji impacted by Tropical Cyclone Yasa. These geometries were digitised by the <a href="https://emergency.copernicus.eu/mapping/list-of-activations-rapid" target="_blank">European Commission Copernicus Emergency Management Service (EMS) - Rapid Mapping Activations</a>. 

We also have a post tropical cyclone event NDVI image computed from Sentinel-2 satellite data. We wish to i) compute the average NDVI value of pixels inside each polygon and ii) generate a sample of polygons where there was no observed disaster impact and compute the average NDVI values for these no-impact polygons. 

To compute the average NDVI values for all pixels that intersect a polygon geometry we will use a raster-vector operation called **zonal statistics**. 

However, before we get to the stage where we can compute zonal statistics we need to pre-process our polygon geometries. This involves reprojecting the geometries to match the coordinate reference system of the raster data and performing a range of <a href="https://geopandas.org/en/stable/docs/user_guide/geometric_manipulations.html" target="_blank">**geometry operations**</a> to generate a sample of no-impact polygons. <a href="https://py.geocompx.org/04-geometry-operations" target="_blank">**Geometry operations**</a> involve manipulating the shape of geometric objects. 

Geometry operations that manipulate shapes include computing the centroids of polygons (transforming a polygon object to a point), buffering a geometry (increasing the size of a geometry), computing the intersection between two geometries (reducing the area, length, or size of a geometry based on its relationship with another geometry), or combining two geometries by computing their union. It is important to be able to look up different geometry operations in the <a href="https://geopandas.org/en/stable/docs/user_guide/geometric_manipulations.html" target="_blank">GeoPandas</a> documentation. Geometry operations are important parts of data transformation and geoprocessing workflows where raw or input spatial data is converted into a format for subsequent tasks such as statistical analysis, machine learning, or data visualisation.  

## Setup

### Run the labs

You can run the labs locally on your machine or you can use cloud environments provided by Google Colab. **If you're working with Google Colab be aware that your sessions are temporary and you'll need to take care to save, backup, and download your work.**

<a href="https://colab.research.google.com/github/geog3300-agri3003/coursebook/blob/main/docs/notebooks/week-4_practice.ipynb" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### Download data

If you need to download the data for this lab, run the following code snippet. 

In [ ]:
import os
import subprocess

if "data_lab-4_practice" not in os.listdir(os.getcwd()):
    subprocess.run('wget "https://github.com/geog3300-agri3003/lab-data/raw/main/data_lab-4_practice.zip"', shell=True, capture_output=True, text=True)
    subprocess.run('unzip "data_lab-4_practice.zip"', shell=True, capture_output=True, text=True)
    if "data_lab-4_practice" not in os.listdir(os.getcwd()):
        print("Has a directory called data_lab-4_practice been downloaded and placed in your working directory? If not, try re-executing this code chunk")
    else:
        print("Data download OK")

### Working in Colab

If you're working in Google Colab, you'll need to install the required packages that don't come with the colab environment.

In [ ]:
if 'google.colab' in str(get_ipython()):
    !pip install xarray[complete]
    !pip install rioxarray
    !pip install mapclassify
    !pip install rasterio
    !pip install rasterstats

### Import modules

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import pprint 
import plotly.express as px
import plotly.io as pio

from rasterstats import zonal_stats

# setup renderer
if 'google.colab' in str(get_ipython()):
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "jupyterlab"

## Geometry operations

Let's load the vector polygon data that delineates the observed disaster impact following Tropical Cyclone Yasa and visualise it to see it's structure.

In [ ]:
obs_event_path = os.path.join(os.getcwd(), "data_lab-4_practice", "EMSR489_AOI07_GRA_PRODUCT_observedEventA_r1_v1.shp")
obs_event_gdf = gpd.read_file(obs_event_path)
obs_event_gdf.explore()

In [ ]:
obs_event_gdf.head()

We're going to be using these geometries in a **zonal statistics** operation. This is a raster-vector operation where we compute summary statistics for all raster pixels that intersect with a polygon geometry. Here, we'll use a Sentinel-2 NDVI image captured just after Tropical Cyclone Yasa impacted this region. However, this satellite image has the projection system `EPSG:32760`. 

**Can you use the GeoPandas `GeoDataFrame` method `to_crs()` to convert `obs_event_gdf` to `EPSG:32760`? Assign the variable name `obs_event_gdf_32760` to the reprojected `GeoDataFrame`.**

The GeoPandas docs for reprojection are <a href="https://geopandas.org/en/stable/docs/user_guide/projections.html#re-projecting" target="_blank">here</a> if you want to look up how to do this. 

In [ ]:
## ADD CODE HERE ##

<details>
    <summary><b>answer</b></summary>

```python
obs_event_gdf_32760 = obs_event_gdf.to_crs("EPSG:32760")
```
</details>

<p></p>

<details>
    <summary><b>Why do we need the polygon geometries and raster data to be in the same projection system for zonal statistics operations?</b></summary>

To ensure the polygon geometries are intersected with raster pixels that correspond to the same location on the Earth's land surface. 

</details>

<p></p>

Our task is to compute summary statistics for each of the polygon geometries that delineate an area impacted by the tropical cyclone event. However, for comparison, we'd also like to generate some sample polygons of areas without a visible impact. We can do this through a sequence of geoprocessing operations that manipulate geometry objects. 

The first step is to combine our `GeoDataFrame` of many polygon objects into a single multipolygon object. We can do this using the <a href="https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.unary_union.html" target="_blank">`unary_union` property</a> of a `GeoSeries` (i.e. the `geometry` column in a `GeoDataFrame`). The `unary_union` computes the union of many polygon objects. The is a geometry aggregation operation, combining many geometry objects into one. 

Below is a quick visualisation of a multipolygon object. We can use multipolygon objects in a number of ways in our programs. For example, we could use multipolygon objects to conveniently represent a country consisting of many islands as a single feature. However, here we'll use them as they help simplify some of the geometry operations we wish to perform in subsequent tasks.

In [ ]:
import shapely
shapely.geometry.MultiPolygon([
    shapely.geometry.Polygon([(0.8,0.8), (0.9,1.0), (1.0,0.9), (0.9,0.9), (0.8,0.8)]), 
    shapely.geometry.Polygon([(0.0,0.1), (0.2,0.4), (0.4,0.3), (0.0,0.0), (0.0,0.1)])
])

If we print the multipolygon object, you can see the geometries are represented as a nested collection of polygons.

In [ ]:
print(
    shapely.geometry.MultiPolygon([
        shapely.geometry.Polygon([(0.8,0.8), (0.9,1.0), (1.0,0.9), (0.9,0.9), (0.8,0.8)]), 
        shapely.geometry.Polygon([(0.0,0.1), (0.2,0.4), (0.4,0.3), (0.0,0.0), (0.0,0.1)])
    ])
)

Let's quickly inspect the structure of our `GeoDataFrame` with each polygon geometry on its own row. 

In [ ]:
obs_event_gdf_32760.head()

Now, let's compute the `unary_union` of the `geometry` column. It should return to us a `GeoSeries` with one element (a multipolygon object that has aggregated all the polygons in our `GeoDataFrame` into a single feature). 

In [ ]:
obs_event_gdf_union = gpd.GeoSeries(obs_event_gdf_32760["geometry"].unary_union).set_crs("EPSG:32760")
obs_event_gdf_union

Now we have combined our polygons into a single multipolygon object, we can use the <a href="https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.buffer.html" target="">buffer operation</a> to expand the geometries by a specified distance to cover areas outside the locations impacted by Tropical Cyclone Yasa. 

**Can you apply a 100 m buffer to the multipolygon object referenced by `obs_event_gdf_union` and reference it with the variable name `obs_event_gdf_buffer`?**

**Hint: use the GeoPandas docs for the buffer operation.**

In [ ]:
## ADD CODE HERE ##

<details>
    <summary><b>answer</b></summary>

```python
obs_event_gdf_buffer = obs_event_gdf_union.buffer(100)
```
</details>

<p></p>

Next, we'll need to set the CRS for the buffered geometries back to `EPSG:32760`. In the map below, areas where disaster impact following the tropical cyclone was observed are coloured red and adjacent areas where not disaster impact was observed are coloured blue. 

In [ ]:
obs_event_gdf_buffer = obs_event_gdf_buffer.set_crs("EPSG:32760")
m = obs_event_gdf_buffer.explore()
obs_event_gdf_32760.explore(m=m, color="red")

We want to use this data as sample polygons where there was no disaster impact. Therefore, we need to remove the interior areas of the polygons which correspond to the locations where disaster impact on the land surface was recorded (these are the red polygons on the map above). We can do this using the GeoPandas <a href="https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.difference.html" target="_blank">`difference()` method</a>; this returns to us geometries of all the locations in one geometry that are not in another geometry.

![](https://github.com/data-analysis-3300-3003/figs/raw/main/week-4-difference.jpg)

**Can you use the `difference()` method of the `GeoDataFrame` `obs_event_gdf_buffer` to remove locations where a disaster event impact was recorded (represented in the `GeoDataFrame` `obs_event_gdf_union`)?**

Assign the result to the variable name `obs_event_no_impact`.

In [ ]:
## ADD CODE HERE ##

<details>
    <summary><b>answer</b></summary>

```python
obs_event_no_impact = obs_event_gdf_buffer.difference(obs_event_gdf_union)
```
</details>

<p></p>

Let's check the result looks sensible. 

In [ ]:
obs_event_no_impact.explore()

<details>
    <summary><b>Why did we compute the buffer and difference operations on a multipolygon (created via the <code>unary_union</code>) instead of using a <code>GeoDataFrame</code> of many polygon geometries?</b></summary>
    
If we computed the buffer operation on many separate polygons, we'd return many larger buffered polygons which would be overlapping in places. This, in itself, is not necessarily an issue. But, we use the buffered polygons in a <code>difference()</code> operation to remove interior areas where disaster impacts occurred. The <code>difference()</code> operation works on a row-by-row basis (i.e. the geometry in row 1 of <code>GeoDataFrame</code> A is compared to the geometry of row 1 in <code>GeoDataFrame</code> B - the geometry in row 1 of <code>GeoDataFrame</code> A is NOT compared to the geometry of row 2 in <code>GeoDataFrame</code> B). This is problematic if the disaster impact area of represented by the geometry in row 2 in <code>GeoDataFrame</code> B overlaps with the geometry in row 1 of <code>GeoDataFrame</code> A. In this case we would not be able to fully remove areas where disaster impact was recorded. However, by comparing two multipolygon objects on a row-by-row basis (there is just one row in <code>GeoDataFrame</code> A and B) we can ensure all areas where disaster impact was recorded are removed. 
</details>

Now, let's convert the `GeoSeries` multipolygon object into a `GeoSeries` of many individual polygon objects using the <a href="https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.explode.html" target="_blank">`explode` method</a>. This operation takes each of the polygons that comprise the composite multipolygon feature and returns them as a `GeoSeries` of separate polygon features. 

<details>
    <summary><b>Why have we used the <code>explode</code> method to split a multipolygon representation of areas not impacted by the disaster event into separate polygons?</b></summary>
We will use these polygons to capture the NDVI signal of areas not impacted by Tropical Cyclone Yasa. If we compute the mean NDVI value for all pixels that intersect with the multipolygon object we'll get a single value for the entire region of interest. This single mean NDVI value might mask lots of spatial variation in the NDVI signal of not impacted areas; computing the NDVI value for many smaller polygons can provide information on variation in the NDVI values of not impacted areas across the scene. Also, if we have many geometries representing not impacted areas which we can compute mean NDVI values for, we can compare these NDVI values with the mean NDVI of nearby polygons where there was a disaster impact. This might give us a more accurate indication of the impact of the Tropical Cyclone on vegetation cover as we're more likely to be comparing similar land covers or exposure to cyclone event.  
</details>

In [ ]:
obs_event_no_impact

In [ ]:
obs_event_no_impact = obs_event_no_impact.explode(ignore_index=True)
obs_event_no_impact

Finally, let's do some `GeoDataFrame` subsetting and concatenation to combine polygons corresponding to disaster event impact with the no impact polygons.

First, let's make the `obs_event_no_impact` `GeoSeries` a `GeoDataFrame` with an `obj_desc` column to match the observed disaster event impact `GeoDataFrame`. 

In [ ]:
obs_event_no_impact_id = pd.DataFrame({"obj_desc": ["no impact" for x in range(0,35)]})
obs_event_no_impact = gpd.GeoDataFrame(obs_event_no_impact_id, geometry=obs_event_no_impact, crs="EPSG:32760")
obs_event_no_impact.head()

Next, let's just get the `obj_desc` and the `geometry` columns from `obs_event_gdf_32760` and concatenate the two `GeoDataFrames` (i.e. stack them on top of each other row wise).

<details>
    <summary><b>Why are we subsetting out the the <code>obj_desc</code> and <code>geometry</code> columns?</b></summary>
We are subsetting these two columns to match the columns in the <code>obs_event_no_impact</code> <code>GeoDataFrame</code>. This allows us to stack the <code>DataFrame</code>s on top of each other row wise with the columns matching up. 
</details>

In [ ]:
obs_event_gdf_32760 = obs_event_gdf_32760.loc[:, ["obj_desc", "geometry"]]
sample_polygons = pd.concat([obs_event_gdf_32760, obs_event_no_impact], axis=0)
sample_polygons

## Zonal stats

We now have 269 sample polygons of locations affected by Tropical Cyclone Yasa and those without visible impacts. We want to compute the average NDVI values across all pixels that intersect each polygon. This is a **zonal statistics operation**. 

We can use the `zonal_stats()` function from the <a href="https://pythonhosted.org/rasterstats/manual.html#statistics" target="_blank">rasterstats package</a> to perform zonal statistics in Python. 

The `zonal_stats()` function takes in vector data of geometries that zonal statistics are computed for as its first argument (or a file path to a vector file), the path to a raster dataset (i.e. a GeoTIFF file) as its second argument, and optionally a list of statistics to compute to the `stats` argument. You can find the full list of parameters for the `zonal_stats()` function in the package documentation.

**Can you look up what the `all_touched` parameter of the `zonal_stats()` function is used to specify in the <a href="https://pythonhosted.org/rasterstats/manual.html#rasterization-strategy" target="_blank">rasterstats docs</a>?**

We have a GeoTIFF file of NDVI values computed from a Sentinel-2 image captured just after Tropical Cyclone Yasa at the file path `os.path.join(os.getcwd(), "data_lab-4_practice", "ndvi_post_yasa_int16.tif")`.

Let's use it to compute the average NDVI value for each of the polygons in the `GeoDataFrame` `sample_polygons`. `zonal_stats` returns to us a dictionary object for each polygon with the summary statistic label as a key and the computed statistic as the value. 

In [ ]:
ndvi_zonal_stats = zonal_stats(sample_polygons, os.path.join(os.getcwd(), "data_lab-4_practice", "ndvi_post_yasa_int16.tif"), stats=["mean"], all_touched=True)
pprint.pprint(f"print the mean NDVI for the first 10 polygons: {ndvi_zonal_stats[0:10]}")

We can convert the list of dictionary objects into a `DataFrame` using the `DataFrame` constructor function. 

In [ ]:
zonal_stats_df = pd.DataFrame(ndvi_zonal_stats)
zonal_stats_df.head()

The NDVI data is in integer format and has been scaled by 1000 (it is cheaper to store integer data than floating point data). However, NDVI values should be between -1 and 1. Let's fix this by dividing the NDVI values by 1000. 

In [ ]:
zonal_stats_df["mean"] = zonal_stats_df["mean"] / 1000
zonal_stats_df.head()

Finally, we can add the column of `mean` NDVI values for each polygon back to our `GeoDataFrame`. This means we can display our polygons and map the fill colour of each polygon to its mean NDVI value. 

In [ ]:
sample_polygons["ndvi"] = zonal_stats_df.loc[:, ["mean"]]
sample_polygons.head()

In [ ]:
sample_polygons.explore(column="ndvi")

<details>
    <summary><b>Can you identify some limitations of this approach to generating a sample of areas not impacted by Tropical Cyclone Yasa?</b></summary>
    
<ul>
    <li>We used the areas directly adjacent to observed impact as our no impact sample. If there is error in how the disaster impact was delineated, it is likely that a disaster impact signal might leak into zones we've labelled as not impacted. A better strategy might be to use sample locations a specified distance from an observed impact.</li>
    <li>There could be differences in NDVI across land covers. If there are different land covers in the impacted area and not impacted area, we could be assigning a difference in NDVI due to land cover differences to disaster impact.</li>
    <li>We could use difference NDVI images. Comparing the change in NDVI before and after disaster event in areas where a disaster impact was observed to areas where it was not.</li>
</ul>
</details>

<p></p>

<details>
    <summary><b>What data transformation and geoprocessing operations could you deploy if you just wanted to focus on the disaster impacts to croplands?</b></summary>
    
You could obtain a land cover map with a cropland class and mask all pixels that are not classified as cropland before computing zonal stats (i.e. set their values <code>NaN</code>).
</details>